## DataCollection

In [1]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Basic Python packages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

## Make repository for Dataset

In [2]:
import os

DATASET_DIR = 'dataset/sponge' 
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created because they already exist')


Directories not created because they already exist


## Controller set up

In [3]:
controller = widgets.Controller(index=0)

display(controller)

Controller()

In [4]:
camera = Camera()

## GUI set up

In [5]:


widget_width = camera.width
widget_height = camera.height

image_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)
target_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)

x_slider = widgets.FloatSlider(min=-1.5, max=1.5, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.6, max=1.6, step=0.001, description='y')
r_slider = widgets.FloatSlider(min=-2.0, max=2.0, step=0.001, description='r')




def display_xy(camera_image):
    image = np.copy(camera_image)
    
    x = x_slider.value 
    y = y_slider.value
    r = r_slider.value
    
    x = int(x * widget_width / 2 + widget_width / 2)
    y = int(y * widget_height / 2 + widget_height / 2)
    r = int(r * widget_height / 4 + widget_height / 4)
    
    
    
    image = cv2.circle(image, (x, y), r, (0, 255, 0),2)
    jpeg_image = bgr8_to_jpeg(image)
    
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

def xy_uuid(x, y, r):
    return 'xy_%03d_%03d_%03d_%s' % (x * widget_width / 2 + widget_width / 2, y * widget_height / 2 + widget_height / 2,r * widget_height / 4 + widget_height / 4, uuid1())

def save_snapshot(change):
    if change['new']:
        uuid = xy_uuid(x_slider.value, y_slider.value,r_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        free_count.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

controller.buttons[5].observe(save_snapshot, names='value')



button_layout = widgets.Layout(width='128px', height='64px')

free_button = widgets.Button(description='add image', button_style='success', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir('dataset/sponge')))

# free_button.on_click(lambda x: save_snapshot)
free_button.on_click(lambda x: save_snapshot(x))

In [6]:
img_box = widgets.HBox([image_widget, target_widget])
button_box = widgets.HBox([free_count, free_button])

In [7]:
widgets.jsdlink((controller.axes[0], 'value'), (x_slider, 'value'))
widgets.jsdlink((controller.axes[1], 'value'), (y_slider, 'value'))
widgets.jsdlink((controller.axes[3], 'value'), (r_slider, 'value'))


DirectionalLink(source=(Axis(value=-1.5259021896696368e-05), 'value'), target=(FloatSlider(value=0.0, descript…

## Lets collect data

In [8]:
display(widgets.HBox([img_box,button_box]), x_slider, y_slider,r_slider)

FloatSlider(value=1.5259021896696368e-05, description='x', max=1.5, min=-1.5, step=0.001)

FloatSlider(value=-1.5259021896696368e-05, description='y', max=1.6, min=-1.6, step=0.001)

FloatSlider(value=-1.5259021896696368e-05, description='r', max=2.0, min=-2.0, step=0.001)

In [ ]:
camera.stop()